In [160]:
import csv
import sys
from datetime import datetime
from typing import Optional, Dict, Any, List
from pymongo import MongoClient, ASCENDING, DESCENDING  
import traceback
import json


### Описание
Это база данных для хранения и поиска книг Арктической лаборатории, где я работаю. 

#### СУБД
MongoDB -- документоориентированная система отлично подходит для хранения информации о книгах. 

#### Структура: 
* Основные документы -- книги. Для них хранится основная информация о названии, городе и годе издания, авторе. Кроме того, каждой книге приписаны тэги по тематикам (например, антропология / языки севера / ономастика)
* * Для книги может не быть какой-то информации (например, часто не бывает года для всяких брошюр, а еще для части книг информация просто не внесена). MongoDB удобна тем, что нулевые значения не хранятся в памяти, для них просто не создается атрибутов. 
* Книги собираются в коллекции по языку и тэгам. 
* Вспомогательная техническая коллекция -- языки. Она устроена иерархически, и ее задача -- смоделировать дерево языков. Далее это будет использовано при поиске по семьям / группам языков. 

Подробно структуру я рассматриваю в процессе создания базы и коллекций.

### Подключение

In [161]:
client = MongoClient('mongodb://localhost:27017/')
db = client['arctic_library'] 

In [162]:
db.books.drop()
db.languages.drop()
db.tags.drop()

### Основной класс

In [ ]:
# дерево языков
LANGUAGE_TREE = json.load(open('language_tree.json', 'r', encoding='utf-8'))

In [164]:
class CSVBook:
    def __init__(self, row: Dict[str, str]):
        self.author = row.get('автор')
        self.title = row.get('название')
        self.publisher_year = row.get('издательство, год')
        self.tags_str = row.get('тэг') or ''
        self.publication_type = row.get('тип')
        self.location = row.get('где')
        self.language_str = row.get('язык') or ''
        self.language_code = row.get('код') or ''

### Функции для обработки CSV

In [165]:
def parse_languages_with_codes(language_str: str, language_code: str) -> Dict[str, Any]:
    """Парсинг языков и кодов"""
    
    result = {
        'languages': [],
        'language_codes': [],
        'language_details': []
    }

    lang_names = []
    if language_str:
        lang_str_clean = language_str.strip('"').strip("'")
        lang_names = [lang.strip() for lang in lang_str_clean.split(',') if lang.strip()]
    
    code_list = []
    if language_code:
        code_str_clean = language_code.strip('"').strip("'")
        code_list = [code.strip() for code in code_str_clean.split(',') if code.strip()]
    
    for i, lang_name in enumerate(lang_names):
        code = None
        # код из соответствующей позиции в CSV
        if i < len(code_list) and code_list[i]:
            code = code_list[i]    
        else:
            print(f"код для {lang_name} не найден")
        
        result['languages'].append(lang_name)
        if code:
            result['language_codes'].append(code)
        result['language_details'].append({
            'name': lang_name,
            'code': code,
            'is_primary': i == 0
        })
    
    return result

In [166]:
def read_csv(filepath: str) -> List[CSVBook]:
    """Чтение CSV"""
    books = []
    
    try:
        with open(filepath, 'r', encoding='utf-8-sig') as file:
            reader = csv.DictReader(file, quotechar='"', quoting=csv.QUOTE_MINIMAL)
   
            for i, row in enumerate(reader, 1):
                if not any(row.values()):
                    continue
                
                cleaned_row = {}
                for key, value in row.items():
                    if key and value and str(value).strip():
                        cleaned_row[key.strip()] = value.strip()
                    elif key:
                        cleaned_row[key.strip()] = None                
                books.append(CSVBook(cleaned_row))
                
    except Exception as e:
        print(f"ошибка чтения CSV: {e}")
    
    return books

### Конвертация в MongoDB

In [167]:
def transform_books(csv_books: List[CSVBook]) -> List[Dict[str, Any]]:
    """Преобразование в MongoDB"""
    transformed = []
    
    for book in csv_books:
        lang_data = parse_languages_with_codes(book.language_str, book.language_code)
        
        book_doc = {
            'author': book.author,
            'title': book.title,
            'publisher_year': book.publisher_year,
            'publication_type': book.publication_type,
            'location': book.location,
            'tags': [tag.strip() for tag in book.tags_str.split(',') if tag.strip()] if book.tags_str else [],
            
            'languages': lang_data['languages'],
            'language_codes': lang_data['language_codes'],
            'language_details': lang_data['language_details'],
            
            'is_available': True,
            'created_at': datetime.now(),
            'language_count': len(lang_data['languages'])
        }
        
        transformed.append(book_doc)
    
    return transformed

### Создание коллекций

In [ ]:
def create_collections(db):
    """Создает коллекции"""
    
    for coll_name in ['languages', 'tags']:
        if coll_name not in db.list_collection_names():
            db.create_collection(coll_name)
    
    stats = {
        'languages': 0, # коллекция языков
        'tags': 0 # коллекция тем
    }

    # языки
    language_map = {}
    
    for book in db.books.find({}, {'language_codes': 1, 'languages': 1, 'title': 1}):
        codes = book.get('language_codes', [])
        names = book.get('languages', [])
        
        for code, name in zip(codes, names):
            if code and name:
                if code not in language_map:
                    language_map[code] = {
                        'name': name,
                        'book_count': 0,
                        'book_titles': []
                    }
                language_map[code]['book_count'] += 1
                language_map[code]['book_titles'].append(book.get('title', ''))
    
    for code, data in language_map.items():
        db.languages.update_one(
            {'code': code},
            {'$set': {
                'code': code,
                'name': data['name'],
                'book_count': data['book_count'],
                'sample_books': data['book_titles'][:5],
                'updated_at': datetime.now()
            }},
            upsert=True
        )
    
    stats['languages'] = len(language_map)
    
    # индекс для быстрого поиска
    existing_indexes = list(db.languages.list_indexes())
    for idx in existing_indexes:
        if idx['name'] != '_id_':
            db.languages.drop_index(idx['name'])
    
    db.languages.create_index([('code', ASCENDING)], unique=True, name='code_unique_idx')
    
    # рубрики (тэги)
    tag_map = {}
    
    for book in db.books.find({}, {'tags': 1, 'title': 1, 'author': 1}):
        tags = book.get('tags', [])
        for tag in tags:
            if tag:  # пропускаем пустые теги
                if tag not in tag_map:
                    tag_map[tag] = {
                        'book_count': 0,
                        'book_titles': [],
                        'languages': set(),
                        'authors': set()
                    }
                tag_map[tag]['book_count'] += 1
                tag_map[tag]['book_titles'].append(book.get('title', ''))
                tag_map[tag]['languages'].update(book.get('language_codes', []))
                if book.get('author'):
                    tag_map[tag]['authors'].add(book.get('author'))
    
    for tag_name, data in tag_map.items():
        db.tags.update_one(
            {'name': tag_name},
            {'$set': {
                'name': tag_name,
                'book_count': data['book_count'],
                'sample_books': data['book_titles'][:3],
                'unique_languages': list(data['languages']),
                'unique_authors': list(data['authors'])[:5],
                'created_at': datetime.now(),
                'updated_at': datetime.now()
            }},
            upsert=True
        )
    
    stats['tags'] = len(tag_map)
    
    # индекс
    db.tags.create_index([('name', ASCENDING)], unique=True, name='tag_name_idx')
    db.tags.create_index([('book_count', DESCENDING)], name='tag_count_idx')
        
    
    return stats

### Главная функция для загрузки всего

In [ ]:
def main():
    try:
        client = MongoClient('mongodb://localhost:27017/')
        client.admin.command('ping')
        print("Подключено!")
        
        db = client['arctic_library']
        
        # читаем csv
        csv_books = read_csv('Арктическая лаборатория_ библиотека.xlsx - for db (1).csv')
        
        if not csv_books:
            print("Нет данных для импорта")
        
        # преобразуем
        books_to_insert = transform_books(csv_books)
        if not books_to_insert:
            print("Нет данных после преобразования")
        
        db.books.delete_many({})
        # импортируем
        result = db.books.insert_many(books_to_insert)
        print(f"Импортировано в MongoDB.")
        
        # коллекции
        stats = create_collections(db)
        
        # индексы для books
        db.books.create_index([('title', ASCENDING)])
        db.books.create_index([('author', ASCENDING)])
        db.books.create_index([('language_codes', ASCENDING)])
        db.books.create_index([('tags', ASCENDING)])

        db.books.create_index([
        ('title', 'text'),
        ('author', 'text'),
        ('languages', 'text'),
        ('tags', 'text'),
        ('publication_type', 'text')], name='text_search_index') # для полнотекстового поиска

        total = db.books.count_documents({})
        print(f"Всего книг: {total}")
    
        # все коллекции
        print(f"\nКоллекции в БД:")
        for coll_name in sorted(db.list_collection_names()):
            count = db[coll_name].count_documents({})
            print(f"  • {coll_name}: {count} документов")

        client.close()
        print("\nГотово.")
        
    except Exception as e:
        print(f"Ошибка {e}")
    

In [170]:
main()

Подключено!
код для английском не найден
код для японском и нивхском языках не найден
Импортировано в MongoDB.
Всего книг: 383

Коллекции в БД:
  • books: 383 документов
  • language_tree: 75 документов
  • languages: 60 документов
  • tags: 37 документов

Готово.


### Функция для красивого вывода результатов

In [ ]:
def print_books_table(books, fields=['title', 'author', 'languages', 'tags'], limit=10):
    """
    Найденные книги в виде таблицы
    
    Args:
        books: список книг (курсор или список документов)
        fields: какие поля показывать
        limit: максимальное количество строк
    """
    
    # курсор в список
    if not isinstance(books, list):
        books = list(books)
    
    if not books:
        print("Книги не найдены")
        return
    
    books = books[:limit]
    
    print(f"\nНайдено {len(books)} книг")
    print("="*80)
    
    headers = ['№', 'Название', 'Автор', 'Языки', 'Теги']
    print(f"{'№':<3} | {'Название':<30} | {'Автор':<20} | {'Языки':<15} | {'Теги'}")
    print("-"*80)
    
    for i, book in enumerate(books, 1):
        title = str(book.get('title', ''))[:28] + '...' if len(str(book.get('title', ''))) > 28 else str(book.get('title', ''))
        author = str(book.get('author', 'Не указан'))[:18] + '...' if len(str(book.get('author', ''))) > 18 else str(book.get('author', 'Не указан'))
        languages = book.get('languages', [])

        if isinstance(languages, list):
            lang_str = ', '.join(str(l) for l in languages[:2])
            if len(languages) > 2:
                lang_str += '...'
        else:
            lang_str = str(languages)[:12]
        tags = book.get('tags', [])
            
        if isinstance(tags, list):
            tag_str = ', '.join(str(t) for t in tags[:2])
            if len(tags) > 2:
                tag_str += '...'
        else:
            tag_str = str(tags)[:20]
        
        print(f"{i:<3} | {title:<30} | {author:<20} | {lang_str:<15} | {tag_str}")
    
    print("="*80)

### Поиск по теме

In [174]:
tags = ["фольклор"]
print_books_table((db.books.find(
    {'tags': {'$in': tags}}
)))


Найдено 10 книг
№   | Название                       | Автор                | Языки           | Теги
--------------------------------------------------------------------------------
1   | Памятники фольклора народов ... | None                 | ненецкие        | фольклор
2   | Образы и сюжеты севернорусск... | О.А. Черепанова      | русский север   | фольклор
3   | Лесные ненцы. Сказания земли... | П.Г. Турутина        | лесной ненецкий | фольклор
4   | Песни нганасан                 | ред. А.Н. Немтушки... | нганасанский    | фольклор
5   | Селькупская мифология          | сост. Г.И. Пелих     | селькупский     | фольклор
6   | Ненецкая литература: Сборник   | сост. Вячеслав Огр... | ненецкие        | фольклор
7   | Фольклор народов Таймыра. Вы... | None                 | ненецкие        | фольклор
8   | Сказка о серой мышке           | Полина Турутина      | ненецкие        | фольклор
9   | Народные песни Ингерманланди... | Cост. В. П. Мироно... | ингерманландский финский | фольклор

#### Статистика по темам: какие языки чаще всего встречаются в разных темах?

In [ ]:
pipeline_languages_in_themes = [
    {'$unwind': '$tags'}, # создает две отдельные записи на каждый тэг из списка тэгов книги
    {'$unwind': '$language_codes'}, # аналогично с языками
    {'$group': {
        '_id': {'tag': '$tags', 'language': '$language_codes'},
        'count': {'$sum': 1} # группируем по комбинации тег + язык и для каждой пары считаем количество документов
    }},
    {'$group': {
        '_id': '$_id.tag',
        'languages': {'$push': {'language': '$_id.language', 'count': '$count'}},
        'total_books': {'$sum': '$count'} # теперь группируем только по тэгу, и для каждого тэга создаем массив языков, которые встретились вместне с ним, считаем их кол-во
    }},
    {'$sort': {'total_books': -1}},
    {'$limit': 5}
]
languages_in_themes = list(db.books.aggregate(pipeline_languages_in_themes))
for theme in languages_in_themes:
    if not theme['_id']:
        continue
    print(f"\n  Тема: {theme['_id']} ({theme['total_books']} книг)")

    # сортируем
    sorted_langs = sorted(theme['languages'], key=lambda x: x['count'], reverse=True)[:3]
    for lang in sorted_langs:
        lang_name = lang['language']
        lang_doc = db.languages.find_one({'code': lang['language']})
        if lang_doc:
            lang_name = lang_doc.get('name', lang['language'])
        print(f"    • {lang_name}: {lang['count']} книг")


  Тема: тексты без перевода (33 книг)
    • ненецкие: 14 книг
    • кильдинский саамский: 10 книг
    • финский: 3 книг

  Тема: тексты с переводом (23 книг)
    • финский: 3 книг
    • чукотский: 3 книг
    • ненецкие: 2 книг

  Тема: антропология (22 книг)
    • ненецкие: 4 книг
    • кольский: 2 книг
    • кильдинский саамский: 2 книг

  Тема: словарь (21 книг)
    • ненецкие: 9 книг
    • финский: 4 книг
    • селькупский: 2 книг

  Тема: грамматическое описание (19 книг)
    • селькупский: 6 книг
    • самодийские: 3 книг
    • мишарский татарский: 2 книг


### Полнотекстовый поиск

In [176]:
# полнотекстовый поиск
print_books_table((db.books.find(
    {'$text': {'$search': 'Иткин'}}
)))


Найдено 1 книг
№   | Название                       | Автор                | Языки           | Теги
--------------------------------------------------------------------------------
1   | Русская морфонология           | И.Б. Иткин           | русский         | теоретическая лингвистика


### Поиск по идиому (семье / группе / языку и т.д.)

Теперь я создаю дополнительную коллекцию для документов другого типа -- языков. 

#### Создание древовидной коллекции -- структура родства языков

In [177]:
def flatten_tree(tree_node, parent_code=None, level=0):
    """дерево --> плоский список"""
    flat_list = []
    
    node_data = {
        'code': tree_node['code'],
        'name': tree_node['name'],
        'type': tree_node['type'],
        'parent_code': parent_code,
        'level': level
    }
    flat_list.append(node_data)
    
    # обрабатываем детей
    if 'children' in tree_node and tree_node['children']:
        for child in tree_node['children']:
            flat_list.extend(flatten_tree(child, tree_node['code'], level + 1))
    
    return flat_list

with open('language_tree.json', 'r', encoding='utf-8') as f:
    tree = json.load(f)

# преобразуем в плоский список
flat_tree = flatten_tree(tree)

existing_indexes = list(db.language_tree.list_indexes())
for idx in existing_indexes:
    if idx['name'] != '_id_':
        db.language_tree.drop_index(idx['name'])
db.language_tree.delete_many({})
result = db.language_tree.insert_many(flat_tree)

# индексы
db.language_tree.create_index([('code', ASCENDING)], unique=True)
db.language_tree.create_index([('parent_code', ASCENDING)])
db.language_tree.create_index([('type', ASCENDING)])


'type_1'

#### Поиск

In [ ]:
NODE_TO_SEARCH = 'rus'

pipeline = [
    # начальный узел
    {'$match': {'code': NODE_TO_SEARCH}},
    
    # рекурсивно находим всех потомков
    {
        '$graphLookup': {
            'from': 'language_tree',
            'startWith': '$code',
            'connectFromField': 'code',
            'connectToField': 'parent_code',
            'as': 'descendants',
            'depthField': 'depth'
        } # поиск по графу
    },
    # получаем все коды (включая сам узел)
    {
        '$addFields': {
            'all_codes': {
                '$concatArrays': [
                    ['$code'],
                    '$descendants.code'
                ]
            }
        }
    },
    # ищем книги с этими кодами
    {
        '$lookup': {
            'from': 'books',
            'let': {'search_codes': '$all_codes'},
            'pipeline': [
                {
                    '$match': {
                        '$expr': {
                            '$gt': [
                                {'$size': {
                                    '$setIntersection': ['$language_codes', '$$search_codes']
                                }},
                                0
                            ]
                        }
                    }
                },
                {'$limit': 10}
            ],
            'as': 'found_books'
        }
    }
]

result = list(db.language_tree.aggregate(pipeline))

node_info = result[0]
print_books_table(node_info['found_books'])


Найдено 7 книг
№   | Название                       | Автор                | Языки           | Теги
--------------------------------------------------------------------------------
1   | Русский язык как иностранный... | Бархударова Е.Л., ... | русский         | РКИ
2   | Русская морфонология           | И.Б. Иткин           | русский         | теоретическая лингвистика
3   | По-русски — с хорошим произн... | Е.Л. Бархударова, ... | русский         | РКИ
4   | Коммуникативные аспекты русс... | Г.А. Золотова        | русский         | теоретическая лингвистика
5   | История русского языка и лин... | В. В. Иванов, А. И... | русский язык    | историческая лингвистика
6   | Историческая грамматика русс... | П. Я. Черных         | русский язык    | историческая лингвистика
7   | Словарь-справочник по русско... | Р. И. Яранцев        | русский         | словарь


К сожалению, не все поля я использую в своих запросах. Конечно, можно искать и по местоположению, и по доступности книги, и по времени ее добавления в библиотеку, и прочая, и прочая. 